In [4]:
import pandas as pd
# load raw data
df = pd.read_csv('File1.txt', sep=' ', names=['Line', 'Date', 'Power'])
print(pd.DataFrame(df))
# builld Days column
df.isnull().sum().sort_values(ascending=False)
df['Date'] = df['Date'].astype('string')
print(df.dtypes)
df['Days'] = [x[3:5] for x in df['Date']]
df['Days'] = df['Days'].astype('int')
print(df)

          Line   Date  Power
0         1392  19503  0.140
1         1392  19504  0.138
2         1392  19505  0.140
3         1392  19506  0.145
4         1392  19507  0.145
...        ...    ...    ...
24465833  1703  73044  0.176
24465834  1703  73045  0.204
24465835  1703  73046  0.178
24465836  1703  73047  0.176
24465837  1703  73048  0.199

[24465838 rows x 3 columns]
Line       int64
Date      string
Power    float64
dtype: object
          Line   Date  Power  Days
0         1392  19503  0.140     3
1         1392  19504  0.138     4
2         1392  19505  0.140     5
3         1392  19506  0.145     6
4         1392  19507  0.145     7
...        ...    ...    ...   ...
24465833  1703  73044  0.176    44
24465834  1703  73045  0.204    45
24465835  1703  73046  0.178    46
24465836  1703  73047  0.176    47
24465837  1703  73048  0.199    48

[24465838 rows x 4 columns]


In [5]:
# drop useless data
df1 = df.drop(df[df['Days']>48].index)
df1.info()
df1

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24461768 entries, 0 to 24465837
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Line    int64  
 1   Date    string 
 2   Power   float64
 3   Days    int64  
dtypes: float64(1), int64(2), string(1)
memory usage: 933.1 MB


,Line,Date,Power,Days
0,1392,19503,0.140,3
1,1392,19504,0.138,4
2,1392,19505,0.140,5
3,1392,19506,0.145,6
4,1392,19507,0.145,7
...,...,...,...,...
24465833,1703,73044,0.176,44
24465834,1703,73045,0.204,45
24465835,1703,73046,0.178,46
24465836,1703,73047,0.176,47


In [12]:
# select the Resident line
import pandas as pd

line_code = pd.read_excel('SME and Residential allocations.xlsx', header=0)
line_code = line_code.loc[:, ['ID', 'Code']]
line_code.sort_values('Code')
line_Resident = line_code[line_code['Code']== 1]
print(line_Resident)


        ID  Code
2     1002     1
3     1003     1
4     1004     1
5     1005     1
6     1006     1
...    ...   ...
6438  7438     1
6440  7440     1
6441  7441     1
6442  7442     1
6443  7443     1

[4225 rows x 2 columns]


In [13]:
# Drop the non-household line
df1 = df1[df1['Line'].isin(line_Resident['ID'])]
df1

,Line,Date,Power,Days
288,1951,19604,0.042,4
289,1951,19605,0.021,5
290,1951,19606,0.021,6
291,1951,19607,0.041,7
292,1951,19543,0.295,43
...,...,...,...,...
24465065,1980,73044,0.010,44
24465066,1980,73045,0.010,45
24465067,1980,73046,0.010,46
24465068,1980,73047,0.010,47


In [14]:
# count the data intergity
from collections import Counter
group_Line = df1['Date'].groupby(df1['Line'])
Date_Number = group_Line.count()
print(Date_Number)
Date_Number_count = Counter(Date_Number)
print(dict(Date_Number_count))
print(len(dict(Date_Number_count)))

Line
1002    25726
1003    25726
1004    25726
1005    25726
1006    25726
        ...  
1992    25726
1993    25726
1994    25726
1996    25726
1997    25726
Name: Date, Length: 651, dtype: int64
{25726: 556, 25678: 86, 25630: 9}
3


In [15]:
# select the Line with relative complete data
Line_Resident=pd.DataFrame(Date_Number)
Line_Resident.columns = ['Timepoint']
Line_Resident = Line_Resident[Line_Resident['Timepoint']==25726]
print(Line_Resident)
index = list(Line_Resident.index)
print(index[::50])

      Timepoint
Line           
1002      25726
1003      25726
1004      25726
1005      25726
1006      25726
...         ...
1992      25726
1993      25726
1994      25726
1996      25726
1997      25726

[556 rows x 1 columns]
[1002, 1084, 1183, 1283, 1381, 1484, 1567, 1656, 1742, 1821, 1904, 1991]


In [17]:
Power_Resident = df1[df1['Line'].isin(index)]
Power_Resident.sort_values(by=['Line', 'Date'])

,Line,Date,Power,Days
171246,1002,19501,0.362,1
171247,1002,19502,0.064,2
171248,1002,19503,0.119,3
171249,1002,19504,0.023,4
171250,1002,19505,0.140,5
...,...,...,...,...
24352169,1997,73044,1.577,44
24352170,1997,73045,1.161,45
24352171,1997,73046,1.010,46
24352172,1997,73047,0.871,47


In [8]:
df_date = Power_Resident.groupby('Date')
Resident_total_power = df_date['Power'].sum().reset_index()
Resident_total_power['Date'] = Resident_total_power['Date'].astype('string')
Resident_total_power['Timepoint'] = [x[3:5] for x in Resident_total_power['Date']]
Resident_total_power['Timepoint'] = Resident_total_power['Timepoint'].astype('int')
Resident_total_power

,Date,Power,Timepoint
0,19501,184.337,1
1,19502,162.016,2
2,19503,140.663,3
3,19504,128.021,4
4,19505,117.371,5
...,...,...,...
25721,73044,449.379,44
25722,73045,424.161,45
25723,73046,411.036,46
25724,73047,407.593,47


In [9]:
# find missing-data
print(Resident_total_power.iloc[12330:12340])
# fill missing date with yesterday and tomorrow data
# missing date:[45202, 45203]
df1 = Resident_total_power.iloc[:12337]
df2 = Resident_total_power.iloc[12337:]
df1 = df1.append([{'Date':45202, 'Power':229.469, 'Timepoint':2}], ignore_index=True)
df1 = df1.append([{'Date':45203, 'Power':192.018, 'Timepoint':3}], ignore_index=True)
print(df1)
Resident_total_power_clean = pd.concat([df1, df2], axis=0).reset_index()
Resident_total_power_clean = Resident_total_power_clean.loc[:, ['Date', 'Power', 'Timepoint']]
print(Resident_total_power_clean)

        Date    Power  Timepoint
12330  45143  431.242         43
12331  45144  397.022         44
12332  45145  346.103         45
12333  45146  317.368         46
12334  45147  327.038         47
12335  45148  276.470         48
12336  45201  229.469          1
12337  45204  192.018          4
12338  45205  171.569          5
12339  45206  156.672          6
        Date    Power  Timepoint
0      19501  184.337          1
1      19502  162.016          2
2      19503  140.663          3
3      19504  128.021          4
4      19505  117.371          5
...      ...      ...        ...
12334  45147  327.038         47
12335  45148  276.470         48
12336  45201  229.469          1
12337  45202  229.469          2
12338  45203  192.018          3

[12339 rows x 3 columns]
        Date    Power  Timepoint
0      19501  184.337          1
1      19502  162.016          2
2      19503  140.663          3
3      19504  128.021          4
4      19505  117.371          5
...      ...     

In [10]:
# read and concat the other featrues
features = pd.read_csv('CER.csv')
features = features.loc[:, ['holidays', 'Hour', 'Days', 'Month', 'Temperature', 'humiduity', 'Wind_speed']]
print(features)
CER_Resident_Power_556 = pd.concat([Resident_total_power_clean, features], axis=1)
CER_Resident_Power_556.to_csv('CER_Resident_Power_556.csv')
CER_Resident_Power_556

       holidays  Hour  Days  Month  Temperature  humiduity  Wind_speed
0             0   0.0    14      7        11.80       93.0        14.4
1             0   0.5    14      7        11.80       91.0        14.4
2             0   1.0    14      7        11.80       89.0        18.0
3             0   1.5    14      7        11.60       91.0        18.0
4             0   2.0    14      7        11.40       93.0        10.8
...         ...   ...   ...    ...          ...        ...         ...
25723         1  21.5    31     12         5.55       87.5         7.2
25724         1  22.0    31     12         5.60       86.0        10.8
25725         1  22.5    31     12         5.60       85.0        10.8
25726         1  23.0    31     12         5.60       84.0        10.8
25727         1  23.5    31     12         5.60       84.0        10.8

[25728 rows x 7 columns]


,Date,Power,Timepoint,holidays,Hour,Days,Month,Temperature,humiduity,Wind_speed
0,19501,184.337,1,0,0.0,14,7,11.80,93.0,14.4
1,19502,162.016,2,0,0.5,14,7,11.80,91.0,14.4
2,19503,140.663,3,0,1.0,14,7,11.80,89.0,18.0
3,19504,128.021,4,0,1.5,14,7,11.60,91.0,18.0
4,19505,117.371,5,0,2.0,14,7,11.40,93.0,10.8
...,...,...,...,...,...,...,...,...,...,...
25723,73044,449.379,44,1,21.5,31,12,5.55,87.5,7.2
25724,73045,424.161,45,1,22.0,31,12,5.60,86.0,10.8
25725,73046,411.036,46,1,22.5,31,12,5.60,85.0,10.8
25726,73047,407.593,47,1,23.0,31,12,5.60,84.0,10.8


In [32]:
import seaborn as sns
import matplotlib.pyplot as plt
# select several household
# ID:[1002, 1139, 1283, 1381, 1484, 1580, 1656, 1742, 1884, 1990]

power_household_1002 = Power_Resident[Power_Resident['Line']==1742].sort_values(by=['Date']).reset_index().drop(['index'], axis=1)
power_household_1002 = power_household_1002.rename(columns={'Days':'Timepoint'})
print(power_household_1002)

# find the missing data
print(power_household_1002.iloc[12330:12340])
df1 = power_household_1002.iloc[:12337]
df2 = power_household_1002.iloc[12337:]
df1 = df1.append([{'Date':45202, 'Power':power_household_1002.iloc[12336,2], 'Timepoint':2}], ignore_index=True)
df1 = df1.append([{'Date':45203, 'Power':power_household_1002.iloc[12337,2], 'Timepoint':3}], ignore_index=True)
Household_1002_clean = pd.concat([df1, df2], axis=0).reset_index()
Household_1002_clean = Household_1002_clean.loc[:, ['Date', 'Power', 'Timepoint']]
print(Household_1002_clean)

# concat the features
features = pd.read_csv('CER.csv')
features = features.loc[:, ['holidays', 'Hour', 'Days', 'Month', 'Temperature', 'humiduity', 'Wind_speed']]
Household_1002_power = pd.concat([Household_1002_clean, features], axis=1)
print(Household_1002_power)
Household_1002_power.to_csv('line_1742.csv')
# plot the correlation heatmap
#corr = Household_1002_power.corr()
#plt.figure(figsize=(15,10))
#sns.heatmap(corr, cmap='Blues',annot=True)

       Line   Date  Power  Timepoint
0      1742  19501  0.123          1
1      1742  19502  0.157          2
2      1742  19503  0.163          3
3      1742  19504  0.129          4
4      1742  19505  0.119          5
...     ...    ...    ...        ...
25721  1742  73044  0.138         44
25722  1742  73045  0.141         45
25723  1742  73046  0.176         46
25724  1742  73047  0.165         47
25725  1742  73048  0.127         48

[25726 rows x 4 columns]
       Line   Date  Power  Timepoint
12330  1742  45143  0.285         43
12331  1742  45144  0.289         44
12332  1742  45145  0.287         45
12333  1742  45146  0.281         46
12334  1742  45147  0.283         47
12335  1742  45148  0.315         48
12336  1742  45201  0.285          1
12337  1742  45204  0.288          4
12338  1742  45205  0.289          5
12339  1742  45206  1.338          6
        Date  Power  Timepoint
0      19501  0.123          1
1      19502  0.157          2
2      19503  0.163          3

In [21]:
df1.iloc[12336, 2]

0.175

In [12]:
CER_Resident_Power_556


,Date,Power,Timepoint,holidays,Hour,Days,Month,Temperature,humiduity,Wind_speed
0,19501,184.337,1,0,0.0,14,7,11.80,93.0,14.4
1,19502,162.016,2,0,0.5,14,7,11.80,91.0,14.4
2,19503,140.663,3,0,1.0,14,7,11.80,89.0,18.0
3,19504,128.021,4,0,1.5,14,7,11.60,91.0,18.0
4,19505,117.371,5,0,2.0,14,7,11.40,93.0,10.8
...,...,...,...,...,...,...,...,...,...,...
25723,73044,449.379,44,1,21.5,31,12,5.55,87.5,7.2
25724,73045,424.161,45,1,22.0,31,12,5.60,86.0,10.8
25725,73046,411.036,46,1,22.5,31,12,5.60,85.0,10.8
25726,73047,407.593,47,1,23.0,31,12,5.60,84.0,10.8


In [29]:
#Spring
CER_Residents_Spring = CER_Resident_Power_556[(CER_Resident_Power_556['Month']>=3) 
                                              & (CER_Resident_Power_556['Month']<=5)]
#Summer
CER_Residents_Summer = CER_Resident_Power_556[(CER_Resident_Power_556['Month']>=6) 
                                              & (CER_Resident_Power_556['Month']<=8)]
#Autumn
CER_Residents_Autumn = CER_Resident_Power_556[(CER_Resident_Power_556['Month']>=9) 
                                              & (CER_Resident_Power_556['Month']<=11)]
#Winter
CER_Residents_Winter = CER_Resident_Power_556[(CER_Resident_Power_556['Month']<=2) 
                                              | (CER_Resident_Power_556['Month']>=12)]

# worktime
CER_Residents_work = CER_Resident_Power_556[(CER_Resident_Power_556['Timepoint']>18)
                                             &(CER_Resident_Power_556['Timepoint']<=34)
                                             &(CER_Resident_Power_556['holidays'] == 0)]
CER_Residents_work
CER_Residents_work.to_csv('CER_Residents_worktime.csv')